# Bollinger Band Strategy Test Multiple
This is a backtesting program that uses real time data to backtest trading strategy. This program takes multiple tickers in CSV format and multiple periods and backtests its performance over a period. It uses Bollinger Bands and Relative Strength Indicators as entry and exit indicators. The results show us the profit n loss, number of trades executed, number of profitable trades for all availabe symbols. This program can be used to make an educated bet in the market. 


In [1]:
import time
import datetime
import pathlib
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf
from pathlib import Path

In [2]:
#variables
sl_sd = 1.5
tp_sd = 2
ub_sd = 2
lb_sd = 2
#create and fill date array
date_list = []
date = pd.read_csv('30_rand_dates.csv', usecols=['Dates'])
date_length = len(date['Dates'].values)
for j in range(0,date_length):
    date_list.append(date['Dates'].values[j])
#create and fill symbol array
    symbol_list =[]
symbol = pd.read_csv('data.csv', skipinitialspace=True, usecols=['symbol'])
length = len(symbol['symbol'].values)
for i in range(0,length):
    symbol_list.append(symbol['symbol'].values[i])

In [3]:

for k in date_list:
    data = {'symbol':[],'pnl':[]}
    pnl_result = pd.DataFrame(data)
    
    for i in symbol_list:
        d_date = datetime.datetime.strptime(k, "%m/%d/%Y").date()
        start = d_date.strftime("%Y-%m-%d")
        end_date = d_date + datetime.timedelta(days=1)
        end = end_date.strftime("%Y-%m-%d")
        
        # Getting Historical Data
        try:
            df = yf.download(i, start=start, end=end, interval = "1m")
        except:
            pass    
            
        #Drop unwanted Columns
        df = df.drop(['High', 'Low', 'Volume'], axis = 1)
        # Convert DataFrame Index to DateTime
        df.reset_index(level=0, inplace=True)
        # Rename columns
        df = df.rename(columns={'Datetime':'time','Open':'open', 'Close':'close'})

        # calculate bollinger bands

        # calculate sma
        df['sma'] = df['close'].rolling(20).mean()

        # calculate standard deviation
        df['sd'] = df['close'].rolling(20).std()

        # calculate lower band
        df['lb'] = df['sma'] - lb_sd * df['sd']

        # calculate upper band
        df['ub'] = df['sma'] + ub_sd * df['sd']

        df.dropna(inplace=True)

        # find signals

        def find_signal(close, lower_band, upper_band):
            if close < lower_band:
                return 'buy'
            elif close > upper_band:
                return 'sell'


        df['signal'] = np.vectorize(find_signal)(df['close'], df['lb'], df['ub'])

    # creating backtest and position classes

        class Position:
            def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
                self.open_datetime = open_datetime
                self.open_price = open_price
                self.order_type = order_type
                self.volume = volume
                self.sl = sl
                self.tp = tp
                self.close_datetime = None
                self.close_price = None
                self.profit = None
                self.status = 'open'

            def close_position(self, close_datetime, close_price):
                self.close_datetime = close_datetime
                self.close_price = close_price
                self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                                else (self.open_price - self.close_price) * self.volume
                self.status = 'closed'

            def _asdict(self):
                return {
                    'open_datetime': self.open_datetime,
                    'open_price': self.open_price,
                    'order_type': self.order_type,
                    'volume': self.volume,
                    'sl': self.sl,
                    'tp': self.tp,
                    'close_datetime': self.close_datetime,
                    'close_price': self.close_price,
                    'profit': self.profit,
                    'status': self.status,
                }


        class Strategy:
            def __init__(self, df, starting_balance, volume):
                self.starting_balance = starting_balance
                self.volume = volume
                self.positions = []
                self.data = df

            def get_positions_df(self):
                df = pd.DataFrame([position._asdict() for position in self.positions])
                df['pnl'] = df['profit'].cumsum() + self.starting_balance
                return df

            def add_position(self, position):
                self.positions.append(position)

            def trading_allowed(self):
                for pos in self.positions:
                    if pos.status == 'open':
                        return False

                return True

            def run(self):
                for i, data in self.data.iterrows():

                    if data.signal == 'buy' and self.trading_allowed():
                        sl = data.close - sl_sd * data.sd
                        tp = data.close + tp_sd * data.sd
                        self.add_position(Position(data.time, data.close, data.signal, self.volume, sl, tp))

                    elif data.signal == 'sell' and self.trading_allowed():
                        sl = data.close + sl_sd * data.sd
                        tp = data.close - tp_sd * data.sd
                        self.add_position(Position(data.time, data.close, data.signal, self.volume, sl, tp))

                    for pos in self.positions:
                        if pos.status == 'open':
                            if (pos.sl >= data.close and pos.order_type == 'buy'):
                                pos.close_position(data.time, pos.sl)
                            elif (pos.sl <= data.close and pos.order_type == 'sell'):
                                pos.close_position(data.time, pos.sl)
                            elif (pos.tp <= data.close and pos.order_type == 'buy'):
                                pos.close_position(data.time, pos.tp)
                            elif (pos.tp >= data.close and pos.order_type == 'sell'):
                                pos.close_position(data.time, pos.tp)

                return self.get_positions_df()

        time.sleep(1)
        # run the backtest
        try:
            bollinger_strategy = Strategy(df, 0, 1)
            result = bollinger_strategy.run()
            result = result[result['pnl'].notna()]
            if len(result['pnl'].tail(1).values) == 0:
                data = {'symbol': i, 'pnl': 0}
            else:
                data = {'symbol': i, 'pnl':result['pnl'].tail(1).values[0]}

            pnl_result = pnl_result.append(data, ignore_index = True)
        except:
            pass

    #Create A CSV File
    pnl_result.to_csv("%s.csv"%(start),index=False)
    

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- RALLIS.NS: 1m data not available for startTime=1645074000 and endTime=1645160400. The requested range must be within the last 30 days.


ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set